In [12]:
import os
import shutil
import random

# Original folder structure
base_path = r"C:\Users\acer\Downloads\Processed Images_Fruits"
output_path = r"C:\Users\acer\Downloads\dataset dummy"

# Define qualities and fruits
qualities = ["good quality", "mixed quality", "bad quality"]
fruits = ["Orange", "Apple", "Banana", "Guava", "Lime", "Pomegranate"]  # Add more fruit names here

# Split ratio (90% for train, 10% for validation)
split_ratio = 0.9

# Create the new structure for train and validation
for quality in qualities:
    for fruit in fruits:
        # Create directories for train and validation
        os.makedirs(os.path.join(output_path, "train", f"{quality.replace(' ', '_')}_{fruit}"), exist_ok=True)
        os.makedirs(os.path.join(output_path, "validation", f"{quality.replace(' ', '_')}_{fruit}"), exist_ok=True)

# Move files into the new structure (train/validation)
for quality in qualities:
    quality_path = os.path.join(base_path, quality)
    for fruit in fruits:
        fruit_path = os.path.join(quality_path, fruit)
        train_folder = os.path.join(output_path, "train", f"{quality.replace(' ', '_')}_{fruit}")
        valid_folder = os.path.join(output_path, "validation", f"{quality.replace(' ', '_')}_{fruit}")

        if os.path.exists(fruit_path):
            files = os.listdir(fruit_path)
            random.shuffle(files)
            
            # Split files
            train_files = files[:int(len(files) * split_ratio)]
            valid_files = files[int(len(files) * split_ratio):]
            
            # Move train files
            for img_file in train_files:
                src_file = os.path.join(fruit_path, img_file)
                dst_file = os.path.join(train_folder, img_file)
                shutil.copy(src_file, dst_file)
            
            # Move validation files
            for img_file in valid_files:
                src_file = os.path.join(fruit_path, img_file)
                dst_file = os.path.join(valid_folder, img_file)
                shutil.copy(src_file, dst_file)

print("Dataset restructuring and splitting completed.")


Dataset restructuring and splitting completed.


In [13]:

import tensorflow

from tensorflow import keras

base_model = keras.applications.VGG16(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False)

Freeze Base Model


In [14]:
base_model.trainable = False

Add New Layers


In [21]:
inputs = keras.Input(shape=(224, 224, 3))

x = base_model(inputs, training=False)

#pooling layer 
x = keras.layers.GlobalAveragePooling2D()(x)

#final dense layer
outputs = keras.layers.Dense(18, activation='softmax')(x)


model = keras.Model(inputs,outputs)

In [22]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d_2  (None, 512)               0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_2 (Dense)             (None, 18)                9234      
                                                                 
Total params: 14723922 (56.17 MB)
Trainable params: 9234 (36.07 KB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


Compile the model

In [23]:
model.compile(loss = "categorical_crossentropy" , metrics = ["accuracy"])

Data Augmentation

In [18]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=10,  # randomly rotate images 
    zoom_range=0.1,  # Randomly zoom image
    width_shift_range=0.1,  # randomly shift images horizontally 
    height_shift_range=0.1,  # randomly shift images vertically 
    horizontal_flip=True,  # randomly flip images horizontally
    vertical_flip=False) 

Load Dataset

In [24]:
train_it = datagen.flow_from_directory(r'C:\Users\acer\Downloads\dataset dummy\train', 
                                       target_size=(224, 224), 
                                       color_mode='rgb', 
                                       class_mode="categorical")
valid_it = datagen.flow_from_directory(r'C:\Users\acer\Downloads\dataset dummy\validation', 
                                      target_size=(224, 224), 
                                      color_mode='rgb', 
                                      class_mode="categorical")

Found 19526 images belonging to 18 classes.
Found 19526 images belonging to 18 classes.


Train the Model

In [27]:
import matplotlib.pyplot as plt
history = model.fit(
    train_it,
    validation_data=valid_it,
    steps_per_epoch=train_it.samples/train_it.batch_size,
    validation_steps=valid_it.samples/valid_it.batch_size,
    epochs=20
)


# Plot training & validation accuracy values
plt.figure(figsize=(12, 6))

# Accuracy plot
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='validation accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Loss plot
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='validation loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()


Epoch 1/20
251/610 [===========>..................] - ETA: 6:34 - loss: 1.4626 - accuracy: 0.6938

KeyboardInterrupt: 

Evaluate

In [ ]:
model.evaluate(valid_it, steps=valid_it.samples/valid_it.batch_size)